In [1]:
import pandas as pd
import micom
from gurobipy import *
import numpy as np
from micom import load_pickle
from micom.media import minimal_medium
from micom.workflows import workflow

import cobra
# https://resendislab.github.io/micom/media.html

In [2]:
#Load pickle

#In this case we only use one model to turn off each reaction and see what happends

com = load_pickle("./models/community_6_5_12.pickle")

Academic license - for non-commercial use only - expires 2021-07-16
Using license file C:\Users\santi\gurobi\gurobi.lic
Read LP format model from file C:\Users\santi\AppData\Local\Temp\tmp18_0jwyx.lp
Reading time = 0.17 seconds
: 3213 rows, 7851 columns, 30797 nonzeros


In [3]:
#This next section is meant to remove each reaction and check for biomass production

In [13]:
sol = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
sol

,abundance,growth_rate,reactions,metabolites
compartments,,,,
A,0.090226,0.005407,1512,1217
B,0.390977,0.023432,872,713
C,0.518797,0.031095,1416,1154
medium,NaN,NaN,125,125


In [14]:
goal=sol.growth_rate
goalA = sol.members.growth_rate['A']
goalB = sol.members.growth_rate['B']
goalC = sol.members.growth_rate['C']
goal, goalA, goalB, goalC

(0.025781036902063972,
 0.005406608472749433,
 0.023431969608458635,
 0.031094727106122846)

In [16]:
only_med = [0]*len(com.medium)
#print(len(only_med))
c=0
for r in com.reactions:
    if r.id in com.medium:
        only_med[c]=r
        c+=1
        #print(c, end=', ')
        if c==len(only_med):
            break

In [17]:
#Test all variable/non-variable reactions that we got from the 256 sample analysis

In [18]:
shared = {'3HCO3_NAt': 1, 'AGPAT1': 1,'ATPS4m': 1,'CYOOm3': 1,'CYOR_u10m': 1,'EX_hco3_LPAREN_e_RPAREN_': 1,'EX_hco3_b': 1,
 'EX_leu_L_LPAREN_e_RPAREN_': 1,'EX_leu_L_b': 1,'EX_nh4_LPAREN_e_RPAREN_': 1,'EX_nh4_b': 1,'EX_o2_LPAREN_e_RPAREN_': 1,
 'EX_o2_b': 1,'EX_pi_LPAREN_e_RPAREN_': 1,'EX_pi_b': 1,'EX_ser_L_LPAREN_e_RPAREN_': 1,'EX_ser_L_b': 1,'G3PD1': 1,'GPAM_hs': 1,
 'O2t': 1,'O2tm': 1,'PPM': 1,'RE3273C': 1,'RE3301C': 1,'TKT1': 1,'TKT2': 1,'r0838': 1,'r2136': 1}

shared_n = {'EX_urea_LPAREN_e_RPAREN_': 1,'EX_urea_b': 1,'RETNt': 1,'UREAt5': 1,'UREAtm': 1,'r1427': 1,'r2408': 1,'EBP2r': 1,
 'r0839': 1}

shared_reac = {}
for r in com.reactions:
    temp = r.id.split('__')
    if '___' in r.id:
        temp[0]+='_'
        temp[1]=temp[1][1:]
    if temp[0] in shared:
        if temp[0] not in shared_reac:
            shared_reac[temp[0]]=[]
        shared_reac[temp[0]].append(r)
        
shared_n_reac = {}
for r in com.reactions:
    temp = r.id.split('__')
    if '___' in r.id:
        temp[0]+='_'
        temp[1]=temp[1][1:]
    if temp[0] in shared_n:
        if temp[0] not in shared_n_reac:
            shared_n_reac[temp[0]]=[]
        shared_n_reac[temp[0]].append(r)

In [19]:
shared_50=[]
for ra in shared_reac.keys():
    t_u=[0]*len(shared_reac[ra])
    t_l=[0]*len(shared_reac[ra])
    for i,r in enumerate(shared_reac[ra]):
        t_u[i] = r.upper_bound
        t_l[i] = r.lower_bound
        r.upper_bound = 0
        r.lower_bound = 0
    try:
        sol_i = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
        
        if sol_i.growth_rate < goal*0.6 and (sol_i.members.growth_rate['A']< goalA*0.7 or sol_i.members.growth_rate['B']< goalB*0.7 or sol_i.members.growth_rate['C']< goalC*0.7):
            print(ra, end=' - ')
            print('Growth: ', sol_i.growth_rate, sol_i.members.growth_rate['A'], sol_i.members.growth_rate['B'], sol_i.members.growth_rate['C'], end='\n\n')
            #print('Original bounds: ',t_l,t_u, end='\n---------------------------------------------------------------------------------->\n')
            
        if sol_i.growth_rate < goal*0.5:
            shared_50.append(r.id)
    except:
        print('Error',temp, end='\n--------------------------\n\n')
    
    for i,r in enumerate(shared_reac[ra]):
        r.upper_bound = t_u[i]
        r.lower_bound = t_l[i]
        #print('Restored bounds: ',r.lower_bound,r.upper_bound)
    #print('-----------------------\n')
shared_50

EX_leu_L_b - Growth:  0.0 0.0 0.0 0.0

EX_o2_b - Growth:  0.008492235139051788 0.0017802581781830745 0.007717785270517483 0.010243178781223349

EX_pi_b - Growth:  0.0 0.0 0.0 0.0

AGPAT1 - Growth:  0.0 0.0 0.0 0.0

G3PD1 - Growth:  0.0 0.0 0.0 0.0

GPAM_hs - Growth:  0.0 0.0 0.0 0.0

PPM - Growth:  0.0 0.0 0.0 0.0

RE3273C - Growth:  0.0 0.0 0.0 0.0

RE3301C - Growth:  0.0 0.0 0.0 0.0

CYOR_u10m - Growth:  0.014530595583359103 0.0030468127019389604 0.013206188135796796 0.01752587924480525

O2t - Growth:  0.009233816476138594 0.0019358058927345324 0.008391825380840198 0.011137579637038616

EX_leu_L_LPAREN_e_RPAREN_ - Growth:  0.0 0.0 0.0 0.0

EX_o2_LPAREN_e_RPAREN_ - Growth:  0.00849223513907356 0.0017802581825649623 0.007717785270296913 0.010243178780669476

EX_pi_LPAREN_e_RPAREN_ - Growth:  0.0 0.0 0.0 0.0



['EX_leu_L_b__C',
 'EX_o2_b__C',
 'EX_pi_b__C',
 'AGPAT1__C',
 'G3PD1__C',
 'GPAM_hs__C',
 'PPM__C',
 'RE3273C__C',
 'RE3301C__C',
 'O2t__C',
 'EX_leu_L_LPAREN_e_RPAREN___C',
 'EX_o2_LPAREN_e_RPAREN___C',
 'EX_pi_LPAREN_e_RPAREN___C']

In [20]:
shared_n_50=[]
for ra in shared_n_reac.keys():
    t_u=[0]*len(shared_n_reac[ra])
    t_l=[0]*len(shared_n_reac[ra])
    
    for i,r in enumerate(shared_n_reac[ra]):
        t_u[i] = r.upper_bound
        t_l[i] = r.lower_bound
        r.upper_bound = 0
        r.lower_bound = 0
    try:
        sol_i = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
        
        if sol_i.growth_rate < goal*0.6 and (sol_i.members.growth_rate['A']< goalA*0.7 or sol_i.members.growth_rate['B']< goalB*0.7 or sol_i.members.growth_rate['C']< goalC*0.7):
            print(ra, end=' - ')
            print('Growth: ', sol_i.growth_rate, sol_i.members.growth_rate['A'], sol_i.members.growth_rate['B'], sol_i.members.growth_rate['C'], end='\n\n')
            #print('Original bounds: ',t_l,t_u, end='\n---------------------------------------------------------------------------------->\n')
            
        if sol_i.growth_rate < goal*0.5:
            shared_n_50.append(r.id)
    except:
        print('Error',temp, end='\n--------------------------\n\n')
    
    for i,r in enumerate(shared_n_reac[ra]):
        r.upper_bound = t_u[i]
        r.lower_bound = t_l[i]
        #print('Restored bounds: ',r.lower_bound,r.upper_bound)
    #print('-----------------------\n')
shared_n_50

[]

In [21]:
#Only A

In [22]:
A_only={'ATPtm': 1, 'EX_glu_L_LPAREN_e_RPAREN_': 1, 'EX_glu_L_b': 1, 'FADH2ETC': 1, 'FUMm': 1, 'GLUB0AT3tc': 1, 'MALtm': 1, 
        'MDH': 1, 'NADH2_u10m': 1, 'NICRNS': 1, 'NP1': 1, 'OAADC': 1, 'RPE': 1, 'SUCD1m': 1, 'SUCOASm': 1, 'r0249': 1, 
        'r0391': 1, 'r0840': 1, 'r0841': 1, 'ARGN': 1,'ARGNm': 1,'INSTt2r': 1,'ORNTArm': 1,'VALATB0tc': 1,'r0821': 1,'r1144': 1,
        'r2084': 1, 'r2407': 1}

pop_reac = {}
for r in com.reactions:
    temp = r.id.split('__')
    if '___' in r.id:
        temp[0]+='_'
        temp[1]=temp[1][1:]
    if temp[0] in A_only:
        if temp[0] not in pop_reac:
            pop_reac[temp[0]]=[]
        pop_reac[temp[0]].append(r)
        
pop_50=[]
for ra in pop_reac.keys():
    t_u=[0]*len(pop_reac[ra])
    t_l=[0]*len(pop_reac[ra])
    
    for i,r in enumerate(pop_reac[ra]):
        t_u[i] = r.upper_bound
        t_l[i] = r.lower_bound
        r.upper_bound = 0
        r.lower_bound = 0
    try:
        sol_i = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
        
        if sol_i.growth_rate < goal*0.6 and (sol_i.members.growth_rate['A']< goalA*0.7 or sol_i.members.growth_rate['B']< goalB*0.7 or sol_i.members.growth_rate['C']< goalC*0.7):
            print(ra,end=' - ')
            print('Growth: ', sol_i.growth_rate, sol_i.members.growth_rate['A'], sol_i.members.growth_rate['B'], sol_i.members.growth_rate['C'], end='\n\n')
            #print('Original bounds: ',t_l,t_u, end='\n---------------------------------------------------------------------------------->\n')
            
        if sol_i.growth_rate < goal*0.5:
            pop_50.append(r.id)
    except:
        print('Error',temp, end='\n--------------------------\n\n')
    
    for i,r in enumerate(pop_reac[ra]):
        r.upper_bound = t_u[i]
        r.lower_bound = t_l[i]
        #print('Restored bounds: ',r.lower_bound,r.upper_bound)
    #print('-----------------------\n')
pop_50

NICRNS - Growth:  0.0 0.0 0.0 0.0

NP1 - Growth:  0.0 0.0 0.0 0.0

r0391 - Growth:  0.0 0.0 0.0 0.0



['NICRNS__C', 'NP1__C', 'r0391__C']

In [35]:
#Only B
A_only={'EX_asp_L_LPAREN_e_RPAREN_': 1, 'EX_asp_L_b': 1, 'FBA': 1, 'LEUB0AT3tc': 1, 'PROtm': 1, 'SUCCt2m': 1, 'TPI': 1, 
        'XYLK': 1, 'XYLTD_Dr': 1, 'r0191': 1, 'r0407': 1, 'r0408': 1, 'r0911': 1, 'r0941': 1, 'r1143': 1, 'ACt2m': 1,
        'ACt2r': 1,'AGMTm': 1,'ARGDCm': 1,'ARGtm': 1,'CYSB0AT3tc': 1,'DM_Lcystin': 1,'D_LACt2': 1,'EX_Lcystin_b': 1,
        'EX_ac_LPAREN_e_RPAREN_': 1,'EX_ac_b': 1,'EX_lac_D_LPAREN_e_RPAREN_': 1,'EX_lac_D_b': 1,'EX_ptrc_LPAREN_e_RPAREN_': 1,
        'EX_ptrc_b': 1,'ILEB0AT2tc': 1,'LDH_D': 1,'LEUB0AT2tc': 1,'PTCRTD': 1,'r0027': 1,'r0062': 1,'r0330': 1,'r0907': 1,
        'r2000': 1,'r2093': 1,'r2101': 1,'r2105': 1,'r2114': 1,'r2405': 1,'r2416': 1}

pop_reac = {}
for r in com.reactions:
    temp = r.id.split('__')
    if '___' in r.id:
        temp[0]+='_'
        temp[1]=temp[1][1:]
    if temp[0] in A_only:
        if temp[0] not in pop_reac:
            pop_reac[temp[0]]=[]
        pop_reac[temp[0]].append(r)
        
pop_50=[]
for ra in pop_reac.keys():
    t_u=[0]*len(pop_reac[ra])
    t_l=[0]*len(pop_reac[ra])
    #print(ra)
    for i,r in enumerate(pop_reac[ra]):
        t_u[i] = r.upper_bound
        t_l[i] = r.lower_bound
        r.upper_bound = 0
        r.lower_bound = 0
    try:
        sol_i = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
        
        if sol_i.growth_rate < goal*0.6 and (sol_i.members.growth_rate['A']< goalA*0.7 or sol_i.members.growth_rate['B']< goalB*0.7 or sol_i.members.growth_rate['C']< goalC*0.7):
            print(ra)
            print('Growth: ', sol_i.growth_rate, sol_i.members.growth_rate['A'], sol_i.members.growth_rate['B'], sol_i.members.growth_rate['C'],end='\n\n')
            #print('Original bounds: ',t_l,t_u, end='\n---------------------------------------------------------------------------------->\n')
            
        if sol_i.growth_rate < goal*0.5:
            pop_50.append(r.id)
    except:
        print('Error',temp, end='\n--------------------------\n\n')
    
    for i,r in enumerate(pop_reac[ra]):
        r.upper_bound = t_u[i]
        r.lower_bound = t_l[i]
        #print('Restored bounds: ',r.lower_bound,r.upper_bound)
    #print('-----------------------\n')
pop_50

EX_Lcystin_b
EX_ac_b
EX_asp_L_b
EX_lac_D_b
EX_ptrc_b
AGMTm
ARGDCm
FBA
LDH_D
TPI
XYLK
XYLTD_Dr
r0027
r0191
r0330
r0407
r0408
ACt2m
ACt2r
D_LACt2
PROtm
SUCCt2m
r0907
r0911
r0941
r1143
r2093
r2101
r2114
r2405
r2416
PTCRTD
DM_Lcystin
EX_ac_LPAREN_e_RPAREN_
EX_asp_L_LPAREN_e_RPAREN_
EX_lac_D_LPAREN_e_RPAREN_
EX_ptrc_LPAREN_e_RPAREN_
ILEB0AT2tc
LEUB0AT2tc
r0062
ARGtm
r2000
r2105
CYSB0AT3tc
LEUB0AT3tc


[]

In [34]:
#Only C
A_only={'ACCOACm': 1, 'ADSL1': 1, 'ADSL2': 1, 'ADSS': 1, 'AICART': 1, 'AIRCr_PRASCS': 1, 'ALATA_L': 1, 'ARTPLM3': 1, 'ASNS1': 1,
        'C14STRr': 1, 'C3STDH1Pr': 1, 'C3STKR2r': 1, 'C4STMO1r': 1, 'C4STMO2Pr': 1, 'CDIPTr': 1, 'CHOLtu': 1, 'CHSTEROLtrc': 1, 
        'CLS_hs': 1, 'CTPS1': 1, 'CYStec': 1, 'DAGK_hs': 1, 'DATPtn': 1, 'DCTPtn': 1, 'DGTPtn': 1, 'DHCR72r': 1, 'DHCRD1': 1, 
        'DMATT': 1, 'DNDPt19m': 1, 'DPMVDc': 1, 'DSAT': 1, 'DTMPK': 1, 'DTTPtn': 1, 'EX_biomass_c': 1, 'EX_chol_LPAREN_e_RPAREN_': 1, 
        'EX_chol_b': 1, 'EX_cys_L_LPAREN_e_RPAREN_': 1, 'EX_cys_L_b': 1, 'EX_inost_LPAREN_e_RPAREN_': 1, 'EX_inost_b': 1, 
        'EX_lys_L_LPAREN_e_RPAREN_': 1, 'EX_lys_L_b': 1, 'EX_met_L_LPAREN_e_RPAREN_': 1, 'EX_met_L_b': 1, 'EX_phe_L_LPAREN_e_RPAREN_': 1, 
        'EX_phe_L_b': 1, 'EX_thr_L_LPAREN_e_RPAREN_': 1, 'EX_thr_L_b': 1, 'EX_trp_L_LPAREN_e_RPAREN_': 1, 'EX_trp_L_b': 1, 
        'EX_tyr_L_LPAREN_e_RPAREN_': 1, 'EX_tyr_L_b': 1, 'FAS100COA': 1, 'FAS120COA': 1, 'FAS140COA': 1, 'FAS160COA': 1, 
        'FAS80COA_L': 1, 'FDH': 1, 'FORtr': 1, 'FRDPtcr': 1, 'G5SADrm': 1, 'GARFT': 1, 'GHMT2r': 1, 'GK1': 1, 'GLUDym': 1, 
        'GLUPRT': 1, 'GRTT': 1, 'HMGCOARc': 1, 'IMPC': 1, 'INSTt4': 1, 'IPDDI': 1, 'LNSTLSr': 1, 'LSTO2r': 1, 'METtec': 1, 
        'MEVK1c': 1, 'NDPK1': 1, 'NDPK1m': 1, 'O2ter': 1, 'OMPDC': 1, 'ORPT': 1, 'P5CRm': 1, 'PE_HStm': 1, 'PGPPT': 1, 
        'PGPP_hs': 1, 'PHEtec': 1, 'PIt2m': 1, 'PMEVKc': 1, 'PRAGSr': 1, 'PRFGS': 1, 'PSDm_hs': 1, 'PSFLIPm': 1, 'PSSA1_hs': 1, 
        'RE2954C': 1, 'RNDR1': 1, 'RNDR4': 1, 'SERPT': 1, 'SMS': 1, 'SQLEr': 1, 'SQLSr': 1, 'TMDS': 1, 'TRDR': 1, 'TYRt': 1, 
        'UMPK4': 1, 'URIDK2m': 1, 'biomass_DNA': 1, 'biomass_RNA': 1, 'biomass_carbohydrate': 1, 'biomass_lipid': 1, 
        'biomass_other': 1, 'biomass_protein': 1, 'biomass_reaction': 1, 'r0280': 1, 'r0431': 1, 'r0432': 1, 'r0472': 1, 
        'r0474': 1, 'r0666': 1, 'r0781': 1, 'r0801': 1, 'r0818': 1, 'r0853': 1, 'r1005': 1, 'r2526': 1, 'r2534': 1,'ACACtx': 1,'COAtp': 1,
        'CRNCARtp': 1,'CSNATp': 1,'CSNATr': 1, 'FUMTSULtm': 1, 'GALt2_2': 1, 'GLYSNAT5tc': 1, 'HMGCOAtx': 1, 'HMGLx': 1,'KCC2t': 1,
        'L_LACt2r': 1,'MALTSULtm': 1,'NAt5': 1,'PGI': 1,'RDH1': 1,'RDH1a': 1,'RDH2': 1,'RDH2a': 1,'RDH3': 1,'RDH3a': 1,
        'TRPB0AT3tc': 1,'r1380': 1,'r1992': 1,'r2086': 1,'r2473': 1,'r2521': 1}

pop_reac = {}
for r in com.reactions:
    temp = r.id.split('__')
    if '___' in r.id:
        temp[0]+='_'
        temp[1]=temp[1][1:]
    if temp[0] in A_only:
        if temp[0] not in pop_reac:
            pop_reac[temp[0]]=[]
        pop_reac[temp[0]].append(r)
        
pop_50=[]
for ra in pop_reac.keys():
    t_u=[0]*len(pop_reac[ra])
    t_l=[0]*len(pop_reac[ra])
    print(ra)
    for i,r in enumerate(pop_reac[ra]):
        t_u[i] = r.upper_bound
        t_l[i] = r.lower_bound
        r.upper_bound = 0
        r.lower_bound = 0
    try:
        sol_i = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
        
        if sol_i.growth_rate < goal*0.6 and (sol_i.members.growth_rate['A']< goalA*0.7 or sol_i.members.growth_rate['B']< goalB*0.7 or sol_i.members.growth_rate['C']< goalC*0.7):
            print('Growth: ', sol_i.growth_rate, sol_i.members.growth_rate['A'], sol_i.members.growth_rate['B'], sol_i.members.growth_rate['C'], end='\n\n')
            #print('Original bounds: ',t_l,t_u, end='\n---------------------------------------------------------------------------------->\n')
            
        if sol_i.growth_rate < goal*0.5:
            pop_50.append(r.id)
    except:
        print('Error',temp, end='\n--------------------------\n\n')
    
    for i,r in enumerate(pop_reac[ra]):
        r.upper_bound = t_u[i]
        r.lower_bound = t_l[i]
        #print('Restored bounds: ',r.lower_bound,r.upper_bound)
    #print('-----------------------\n')
pop_50

EX_biomass_c
Growth:  0.0 0.0 0.0 0.0

EX_chol_b
Growth:  0.013440467417995153 0.0 0.0 0.025906987263424136

EX_cys_L_b
EX_inost_b
Growth:  0.0 0.0 0.0 0.0

EX_lys_L_b
Growth:  0.0 0.0 0.0 0.0

EX_met_L_b
Growth:  0.0 0.0 0.0 0.0

EX_phe_L_b
Growth:  0.0 0.0 0.0 0.0

EX_thr_L_b
Growth:  0.0 0.0 0.0 0.0

EX_trp_L_b
Growth:  0.0 0.0 0.0 0.0

EX_tyr_L_b
Growth:  0.0 0.0 0.0 0.0

ACCOACm
ADSL1
Growth:  0.0 0.0 0.0 0.0

ADSL2
Growth:  0.0 0.0 0.0 0.0

ADSS
Growth:  0.0 0.0 0.0 0.0

AICART
Growth:  0.0 0.0 0.0 0.0

ALATA_L
ARTPLM3
ASNS1
Growth:  0.0 0.0 0.0 0.0

C14STRr
Growth:  0.0 0.0 0.0 0.0

C3STDH1Pr
C3STKR2r
C4STMO1r
Growth:  0.0 0.0 0.0 0.0

CDIPTr
Growth:  0.0 0.0 0.0 0.0

CLS_hs
Growth:  0.0 0.0 0.0 0.0

CSNATp
CSNATr
CTPS1
DAGK_hs
DHCR72r
DHCRD1
DMATT
DSAT
Growth:  0.0 0.0 0.0 0.0

DTMPK
FAS100COA
Growth:  0.0 0.0 0.0 0.0

FAS120COA
Growth:  0.0 0.0 0.0 0.0

FAS140COA
Growth:  0.0 0.0 0.0 0.0

FAS160COA
Growth:  0.0 0.0 0.0 0.0

FAS80COA_L
Growth:  0.0 0.0 0.0 0.0

FDH
G5SADrm
Grow

['EX_biomass_c__C',
 'EX_inost_b__C',
 'EX_lys_L_b__C',
 'EX_met_L_b__C',
 'EX_phe_L_b__C',
 'EX_thr_L_b__C',
 'EX_trp_L_b__C',
 'EX_tyr_L_b__C',
 'ADSL1__C',
 'ADSL2__C',
 'ADSS__C',
 'AICART__C',
 'ASNS1__C',
 'C14STRr__C',
 'C4STMO1r__C',
 'CDIPTr__C',
 'CLS_hs__C',
 'DSAT__C',
 'FAS100COA__C',
 'FAS120COA__C',
 'FAS140COA__C',
 'FAS160COA__C',
 'FAS80COA_L__C',
 'G5SADrm__C',
 'GLUPRT__C',
 'IMPC__C',
 'LNSTLSr__C',
 'OMPDC__C',
 'ORPT__C',
 'P5CRm__C',
 'PGPP_hs__C',
 'PGPPT__C',
 'PRAGSr__C',
 'PRFGS__C',
 'PSDm_hs__C',
 'RNDR1__C',
 'SERPT__C',
 'SMS__C',
 'SQLEr__C',
 'SQLSr__C',
 'TMDS__C',
 'TRDR__C',
 'r0666__C',
 'r0781__C',
 'HMGCOARc__C',
 'DATPtn__C',
 'DCTPtn__C',
 'DGTPtn__C',
 'FORtr__C',
 'O2ter__C',
 'PE_HStm__C',
 'PSFLIPm__C',
 'CHSTEROLtrc__C',
 'EX_inost_LPAREN_e_RPAREN___C',
 'EX_lys_L_LPAREN_e_RPAREN___C',
 'EX_met_L_LPAREN_e_RPAREN___C',
 'EX_phe_L_LPAREN_e_RPAREN___C',
 'EX_thr_L_LPAREN_e_RPAREN___C',
 'EX_trp_L_LPAREN_e_RPAREN___C',
 'EX_tyr_L_LPAREN_e_RPAR

In [25]:
#check for all medium exchange reactions individually

In [26]:
t_u=0
t_l=0
c=0
only_med_func=[]
only_med_func_great=[]
single_med_50=[]
single_med_50_pop=[]

for r in only_med:
    t_u=r.upper_bound
    t_l=r.lower_bound
    r.upper_bound = 0
    r.lower_bound = 0
    sol_i = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
    
    #print('Zero bounds: ', r.lower_bound, r.upper_bound)
    
    if sol_i.growth_rate < goal*0.75 or sol_i.members.growth_rate['A']< goalA*0.6 or sol_i.members.growth_rate['B']< goalB*0.6 or sol_i.members.growth_rate['C']< goalC*0.6:
        print(c, r.id, end=' - ')
        print('Growth: ', sol_i.growth_rate, sol_i.members.growth_rate['A'], sol_i.members.growth_rate['B'], sol_i.members.growth_rate['C'], end='\n\n')
        #print('Original flux: ', sol.fluxes[r.id].medium)
        #print('Flux: ', sol_i.fluxes[r.id].medium)
        #print('Original bounds: ', t_l, t_u, end='\n--------------------------------------------------->\n')
    #else:
        #print('Original bounds: ', t_l, t_u, end='\n-----------------------------\n')
        
    r.upper_bound = t_u
    r.lower_bound = t_l
    if sol_i.growth_rate < goal*0.75 and sol_i.growth_rate > 0 :
        only_med_func.append(r)
    if sol_i.growth_rate > 0 :
        only_med_func_great.append(r)
    if sol_i.growth_rate <= goal*0.5:
        single_med_50.append(r.id)
    if sol_i.members.growth_rate['A']< goalA*0.5:
        single_med_50_pop.append((r.id, sol_i.members.growth_rate['A']))
    if sol_i.members.growth_rate['B']< goalB*0.5:
        single_med_50_pop.append((r.id, sol_i.members.growth_rate['B']))
    if sol_i.members.growth_rate['C']< goalC*0.5:
        single_med_50_pop.append((r.id, sol_i.members.growth_rate['C']))
        
        
    #print('Restored bounds: ', r.lower_bound, r.upper_bound, end='\n\n')
    c+=1

print(single_med_50)
print(single_med_50_pop)

1 EX_arg_L_m - Growth:  0.0 0.0 0.0 0.0

4 EX_chol_m - Growth:  0.013440467417993533 0.002818156806699766 0.01221534590235543 0.01621110564450012

8 EX_gal_m - Growth:  0.01914044013776383 0.004013734028987523 0.017396180483972873 0.02308568598082153

11 EX_his_L_m - Growth:  0.0 0.0 0.0 0.0

12 EX_ile_L_m - Growth:  0.0 0.0 0.0 0.0

13 EX_inost_m - Growth:  0.0 0.0 0.0 0.0

14 EX_leu_L_m - Growth:  0.0 0.0 0.0 0.0

15 EX_lys_L_m - Growth:  0.0 0.0 0.0 0.0

16 EX_met_L_m - Growth:  0.0 0.0 0.0 0.0

19 EX_o2_m - Growth:  0.008551992534693386 0.0017927923762668713 0.00777210012410917 0.010315250532218927

20 EX_phe_L_m - Growth:  0.0 0.0 0.0 0.0

21 EX_pi_m - Growth:  0.0 0.0 0.0 0.0

29 EX_thr_L_m - Growth:  0.0 0.0 0.0 0.0

30 EX_trp_L_m - Growth:  0.0 0.0 0.0 0.0

31 EX_tyr_L_m - Growth:  0.0 0.0 0.0 0.0

32 EX_val_L_m - Growth:  0.0 0.0 0.0 0.0

['EX_arg_L_m', 'EX_his_L_m', 'EX_ile_L_m', 'EX_inost_m', 'EX_leu_L_m', 'EX_lys_L_m', 'EX_met_L_m', 'EX_o2_m', 'EX_phe_L_m', 'EX_pi_m', 'EX_t

In [ ]:
#Check for pairs of medium exchange reactions removal

In [27]:
t_u=0
t_l=0
pairs_med_50=[]
pairs_med_50_pop=[]
cou=0

for r in only_med_func:
    c=1
    cou+=1
    t_u=r.upper_bound
    t_l=r.lower_bound
    r.upper_bound = 0
    r.lower_bound = 0
    print(cou, end=':\n')
    for r2 in only_med_func[cou:]:
        t_u2=r2.upper_bound
        t_l2=r2.lower_bound
        r2.upper_bound = 0
        r2.lower_bound = 0
        
        sol_i = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
        #print('Zero bounds: ', r.lower_bound, r.upper_bound, r2.lower_bound, r2.upper_bound)
        
        if sol_i.growth_rate < goal*0.6 or sol_i.members.growth_rate['A']< goalA*0.75 or sol_i.members.growth_rate['B']< goalB*0.75 or sol_i.members.growth_rate['C']< goalC*0.75:
            print(c, r.id, r2.id)
            print('Growth: ', sol_i.growth_rate, sol_i.members.growth_rate['A'], sol_i.members.growth_rate['B'], sol_i.members.growth_rate['C'], end='\n\n')
            
            #print('Original flux: ', sol.fluxes[r.id].medium, sol.fluxes[r2.id].medium)
            #print('Flux: ', sol_i.fluxes[r.id].medium, sol_i.fluxes[r2.id].medium)
            #print('Original bounds: ', t_l, t_u, t_l2, t_u2, end='\n--------------------------------------------------->\n')
        #else:
            #print('Original bounds: ', t_l, t_u, t_l2, t_u2, end='\n-----------------------------\n')
            
        if sol_i.growth_rate <= goal*0.5:
            pairs_med_50.append((r.id,r2.id))
        
        if sol_i.members.growth_rate['A']< goalA*0.5 or sol_i.members.growth_rate['B']< goalB*0.5 or sol_i.members.growth_rate['C']< goalC*0.5:
            pairs_med_50_pop.append((r.id,r2.id))
            
            
        r2.upper_bound = t_u2
        r2.lower_bound = t_l2
        #print('Restored bounds: ', r2.lower_bound, r2.upper_bound, end='\n\n')
        c+=1
    r.upper_bound = t_u
    r.lower_bound = t_l
    print('----------------------------------------Restored bounds: ', r.lower_bound, r.upper_bound, end='\n\n')
    
print(pairs_med_50)
pairs_med_50_pop

1:
1 EX_chol_m EX_gal_m
Growth:  0.010099637115481723 0.0021174129856475466 0.009178789411598857 0.012181822097467805

2 EX_chol_m EX_o2_m
Growth:  0.0020028891380746146 0.0004191090847222461 0.0018194713739697235 0.0024165570091064944

----------------------------------------Restored bounds:  -0.1428 10.0

2:
1 EX_gal_m EX_o2_m
Growth:  0.0021277432413245078 0.00044529714198226377 0.0019329541245681244 0.002567140138597603

----------------------------------------Restored bounds:  -0.111 10.0

3:
----------------------------------------Restored bounds:  -0.2 10.0

[('EX_chol_m', 'EX_gal_m'), ('EX_chol_m', 'EX_o2_m'), ('EX_gal_m', 'EX_o2_m')]


[('EX_chol_m', 'EX_gal_m'), ('EX_chol_m', 'EX_o2_m'), ('EX_gal_m', 'EX_o2_m')]

In [28]:
t_u=0
t_l=0
pairs_all_med_50=[]
pairs_all_med_50_pop=[]
for r in only_med_func_great:
    c=1
    t_u=r.upper_bound
    t_l=r.lower_bound
    r.upper_bound = 0
    r.lower_bound = 0
    for r2 in only_med_func_great:
        t_u2=r2.upper_bound
        t_l2=r2.lower_bound
        r2.upper_bound = 0
        r2.lower_bound = 0
        
        sol_i = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
        #print('Zero bounds: ', r.lower_bound, r.upper_bound, r2.lower_bound, r2.upper_bound)
        
        if sol_i.growth_rate < goal*0.6 or sol_i.members.growth_rate['A']< goalA*0.75 or sol_i.members.growth_rate['B']< goalB*0.75 or sol_i.members.growth_rate['C']< goalC*0.75:
            print(c, r.id, r2.id)
            print('Growth: ', sol_i.growth_rate, sol_i.members.growth_rate['A'], sol_i.members.growth_rate['B'], sol_i.members.growth_rate['C'], end='\n\n')
            #print('Original flux: ', sol.fluxes[r.id].medium, sol.fluxes[r2.id].medium)
            #print('Flux: ', sol_i.fluxes[r.id].medium, sol_i.fluxes[r2.id].medium)
            #print('Original bounds: ', t_l, t_u, t_l2, t_u2, end='\n--------------------------------------------------->\n')
        #else:
            #print('Original bounds: ', t_l, t_u, t_l2, t_u2, end='\n-----------------------------\n')
            
        if sol_i.growth_rate <= goal*0.5:
            pairs_all_med_50.append((r.id,r2.id))
            
        if sol_i.members.growth_rate['A']< goalA*0.5 or sol_i.members.growth_rate['B']< goalB*0.5 or sol_i.members.growth_rate['C']< goalC*0.5:
            pairs_all_med_50_pop.append((r.id,r2.id))
            
            
        r2.upper_bound = t_u2
        r2.lower_bound = t_l2
        #print('Restored bounds: ', r2.lower_bound, r2.upper_bound, end='\n\n')
        c+=1
    r.upper_bound = t_u
    r.lower_bound = t_l
    print('----------------------------------------Restored bounds: ', r.lower_bound, r.upper_bound, end='\n\n')

print(pairs_all_med_50_pop)
pairs_all_med_50

4 EX_ala_L_m EX_chol_m
Growth:  0.013413168418733924 0.0028124308184594863 0.012190533220950959 0.016178181070698457

8 EX_ala_L_m EX_gal_m
Growth:  0.01908980574162084 0.004003113392443554 0.017350157732132394 0.0230246172283476

13 EX_ala_L_m EX_o2_m
Growth:  0.008453706950966876 0.0017721768872820843 0.007682766121989223 0.010196710987359596

----------------------------------------Restored bounds:  -0.2246 10.0

4 EX_asp_L_m EX_chol_m
Growth:  0.012607942644591166 0.0026435335797162377 0.01145864523888093 0.015207020411014318

8 EX_asp_L_m EX_gal_m
Growth:  0.017725910170660036 0.003717034397070406 0.01611048209017005 0.021379660289984514

13 EX_asp_L_m EX_o2_m
Growth:  0.006863817985341421 0.0014386954068512493 0.006237679958792037 0.008279189637079163

----------------------------------------Restored bounds:  -0.1514 10.0

4 EX_ca2_m EX_chol_m
Growth:  0.01344046741799607 0.0028181567964724076 0.012215345903095374 0.01621110564572604

8 EX_ca2_m EX_gal_m
Growth:  0.01914044013776

8 EX_na1_m EX_gal_m
Growth:  0.019140440137769346 0.004013734028006788 0.017396180484051095 0.023085685980943776

13 EX_na1_m EX_o2_m
Growth:  0.008551992534691012 0.0017927923746353285 0.007772100124249034 0.01031525053239269

----------------------------------------Restored bounds:  -0.344827575 10.0

4 EX_ncam_m EX_chol_m
Growth:  0.013440467417993318 0.0028181567997237478 0.012215345902857416 0.016211105645334616

8 EX_ncam_m EX_gal_m
Growth:  0.01914044013776578 0.004013734026848669 0.017396180484134757 0.023085685981075268

13 EX_ncam_m EX_o2_m
Growth:  0.008551992534683743 0.001792792371264202 0.007772100124528522 0.010315250532754335

----------------------------------------Restored bounds:  -0.16393442 10.0

1 EX_o2_m EX_ala_L_m
Growth:  0.008453706950966876 0.0017721768872820843 0.007682766121989223 0.010196710987359596

2 EX_o2_m EX_asp_L_m
Growth:  0.006863817985341421 0.0014386954068512493 0.006237679958792037 0.008279189637079163

3 EX_o2_m EX_ca2_m
Growth:  0.00855199253

[('EX_ala_L_m', 'EX_o2_m'),
 ('EX_asp_L_m', 'EX_chol_m'),
 ('EX_asp_L_m', 'EX_o2_m'),
 ('EX_ca2_m', 'EX_o2_m'),
 ('EX_chol_m', 'EX_asp_L_m'),
 ('EX_chol_m', 'EX_gal_m'),
 ('EX_chol_m', 'EX_glu_L_m'),
 ('EX_chol_m', 'EX_o2_m'),
 ('EX_chol_m', 'EX_ser_L_m'),
 ('EX_cl_m', 'EX_o2_m'),
 ('EX_cys_L_m', 'EX_o2_m'),
 ('EX_fol_m', 'EX_o2_m'),
 ('EX_gal_m', 'EX_chol_m'),
 ('EX_gal_m', 'EX_o2_m'),
 ('EX_glu_L_m', 'EX_chol_m'),
 ('EX_glu_L_m', 'EX_o2_m'),
 ('EX_gly_m', 'EX_o2_m'),
 ('EX_na1_m', 'EX_o2_m'),
 ('EX_ncam_m', 'EX_o2_m'),
 ('EX_o2_m', 'EX_ala_L_m'),
 ('EX_o2_m', 'EX_asp_L_m'),
 ('EX_o2_m', 'EX_ca2_m'),
 ('EX_o2_m', 'EX_chol_m'),
 ('EX_o2_m', 'EX_cl_m'),
 ('EX_o2_m', 'EX_cys_L_m'),
 ('EX_o2_m', 'EX_fol_m'),
 ('EX_o2_m', 'EX_gal_m'),
 ('EX_o2_m', 'EX_glu_L_m'),
 ('EX_o2_m', 'EX_gly_m'),
 ('EX_o2_m', 'EX_na1_m'),
 ('EX_o2_m', 'EX_ncam_m'),
 ('EX_o2_m', 'EX_o2_m'),
 ('EX_o2_m', 'EX_pnto_R_m'),
 ('EX_o2_m', 'EX_pydxn_m'),
 ('EX_o2_m', 'EX_pyr_m'),
 ('EX_o2_m', 'EX_ribflv_m'),
 ('EX_o2_m', 'E

In [29]:
#Check for all other community exchange reactions

In [30]:
t_u=0
t_l=0
ex_50=[]
for r in com.reactions:
    if 'EX_' in r.id and '_m' in r.id and '__' not in r.id and r.id not in com.medium:
        if r.lower_bound!=0 or r.upper_bound!=0:
            t_u=r.upper_bound
            t_l=r.lower_bound
            r.upper_bound = 0
            r.lower_bound = 0
            sol_i = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
            #print('Zero bounds: ', r.lower_bound,r.upper_bound)
            if sol_i.growth_rate < goal*0.6 or sol_i.members.growth_rate['A']< goalA*0.8 or sol_i.members.growth_rate['B']< goalB*0.8 or sol_i.members.growth_rate['C']< goalC*0.8:
                print(r.id)
                print('Growth: ', sol_i.growth_rate, sol_i.members.growth_rate['A'], sol_i.members.growth_rate['B'], sol_i.members.growth_rate['C'], end='\n\n')
                #print('Original flux: ', sol.fluxes[r.id].medium)
                #print('Flux: ', sol_i.fluxes[r.id].medium)
                #print('Original bounds: ',r.lower_bound,r.upper_bound, end='\n--------------------------------------------------->\n\n')
            #else:
                #print('Original bounds: ',r.lower_bound,r.upper_bound, end='\n-----------------------------\n\n')
            
            if sol_i.growth_rate < goal*0.5:
                ex_50.append(r.id)
            
            r.upper_bound = t_u
            r.lower_bound = t_l
            
ex_50

[]

In [31]:
#check for all reactions related to the mitochondria

In [38]:
d = {}
for r in com.reactions:
    if 'm__' in r.id:
        temp = r.id.split('__')
        if '___' in r.id:
            temp[0]+='_'
            temp[1]=temp[1][1:]
        if temp[0] not in d:
            d[temp[0]]=[]
        d[temp[0]].append(r)

In [40]:
t_u=0
t_l=0

mito_50=[]

for ra in d:
    t_u=[0]*len(d[ra])
    t_l=[0]*len(d[ra])
    for i,r in enumerate(d[ra]):
        t_u[i] = r.upper_bound
        t_l[i] =r.lower_bound
        r.upper_bound = 0
        r.lower_bound = 0
        #print('Zero bounds: ', r.lower_bound,r.upper_bound)
    try:
        sol_i = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
        
        if sol_i.growth_rate < goal*0.95 or sol_i.members.growth_rate['A']< goalA*0.95 or sol_i.members.growth_rate['B']< goalB*0.95 or sol_i.members.growth_rate['C']< goalC*0.95:
            print(ra)
            print('Growth: ', sol_i.growth_rate, sol_i.members.growth_rate['A'], sol_i.members.growth_rate['B'], sol_i.members.growth_rate['C'],end='\n\n')
            #print('Original bounds: ',t_l,t_u, end='\n--------------------------------------------------->\n')
        #else:
           # print('Original bounds: ',t_l,t_u, end='\n-----------------------------\n')
            
        if sol_i.growth_rate < goal*0.5:
            mito_50.append(r.id)
        
    except:
        print('Error',temp, end='\n--------------------------\n\n')
    
    for i,r in enumerate(d[ra]):
        r.upper_bound = t_u[i]
        r.lower_bound = t_l[i]
        #print('Restored bounds: ',r.lower_bound,r.upper_bound)
    #print('-----------------------\n')
mito_50

ACCOACm
Growth:  0.02422620656758948 0.0 0.0 0.0466968897337086

FUMm
Growth:  0.025781036902063483 0.008387683224148818 0.0 0.04823515400110964

G5SADrm
Growth:  0.0 0.0 0.0 0.0

P5CRm
Growth:  0.0 0.0 0.0 0.0

PDHm
Growth:  0.022944108595584337 0.0048115587973455685 0.02085342107944067 0.02767318589814562

PPAm
Growth:  0.025319485932181824 0.014188139539888452 0.06148527028203543 0.0

ATPS4m
Growth:  0.016038806545677645 0.0033631620689758873 0.014577035311349082 0.019344891005668243

CYOR_u10m
Growth:  0.014530595583359103 0.0030468127019389604 0.013206188135796796 0.01752587924480525

HMGCOAtm
Growth:  0.0251968394665507 0.0 0.023342758924612936 0.030976176345310663

NADH2_u10m
Growth:  0.021344968565777493 0.004476136943521533 0.01939992640974622 0.02574450708827552

O2tm
Growth:  0.01878731142813041 0.003939664788295327 0.017077819529229436 0.022657823406692063

PE_HStm
Growth:  0.0 0.0 0.0 0.0

PSFLIPm
Growth:  0.0 0.0 0.0 0.0

PPItm
Growth:  0.025319485932187497 0.014188139541

['G5SADrm__C', 'P5CRm__C', 'PE_HStm__C', 'PSFLIPm__C']

In [ ]:
#Check for all other reactions 

In [41]:
d = {}
for r in com.reactions:
    if 'm' != str(r.id[len(r.id)-1]):
        temp = r.id.split('__')
        if '___' in r.id:
            temp[0]+='_'
            temp[1]=temp[1][1:]
        if temp[0] not in d:
            d[temp[0]]=[]
        d[temp[0]].append(r)

In [42]:
reac_50=[]
for ra in d:
    t_u=[0]*len(d[ra])
    t_l=[0]*len(d[ra])
    for i,r in enumerate(d[ra]):
        t_u[i] = r.upper_bound
        t_l[i] =r.lower_bound
        r.upper_bound = 0
        r.lower_bound = 0
        #print('Zero bounds: ', r.lower_bound,r.upper_bound)
    try:
        sol_i = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
        
        if sol_i.growth_rate < goal*0.7 and (sol_i.members.growth_rate['A']< goalA*0.9 or sol_i.members.growth_rate['B']< goalB*0.9 or sol_i.members.growth_rate['C']< goalC*0.9):
            print(ra)
            print('Growth: ', sol_i.growth_rate, sol_i.members.growth_rate['A'], sol_i.members.growth_rate['B'], sol_i.members.growth_rate['C'],end='\n\n')
            #print('Original bounds: ',t_l,t_u, end='\n---------------------------------------------------------------------------------->\n')
        #else:
            #print('Original bounds: ',t_l,t_u, end='\n-----------------------------\n')
            
        if sol_i.growth_rate < goal*0.5:
            reac_50.append(r.id)
    except:
        print('Error',temp, end='\n--------------------------\n\n')
    
    for i,r in enumerate(d[ra]):
        r.upper_bound = t_u[i]
        r.lower_bound = t_l[i]
        #print('Restored bounds: ',r.lower_bound,r.upper_bound)
    #print('-----------------------\n')
reac_50

EX_arg_L_b
Growth:  0.0 0.0 0.0 0.0

EX_biomass_c
Growth:  0.0 0.0 0.0 0.0

EX_chol_b
Growth:  0.013440467417995153 0.0 0.0 0.025906987263424136

EX_his_L_b
Growth:  0.0 0.0 0.0 0.0

EX_ile_L_b
Growth:  0.0 0.0 0.0 0.0

EX_inost_b
Growth:  0.0 0.0 0.0 0.0

EX_leu_L_b
Growth:  0.0 0.0 0.0 0.0

EX_lys_L_b
Growth:  0.0 0.0 0.0 0.0

EX_met_L_b
Growth:  0.0 0.0 0.0 0.0

EX_o2_b
Growth:  0.008492235139051788 0.0017802581781830745 0.007717785270517483 0.010243178781223349

EX_phe_L_b
Growth:  0.0 0.0 0.0 0.0

EX_pi_b
Growth:  0.0 0.0 0.0 0.0

EX_thr_L_b
Growth:  0.0 0.0 0.0 0.0

EX_trp_L_b
Growth:  0.0 0.0 0.0 0.0

EX_tyr_L_b
Growth:  0.0 0.0 0.0 0.0

EX_val_L_b
Growth:  0.0 0.0 0.0 0.0

3DSPHR
Growth:  0.0 0.0 0.0 0.0

ADSL1
Growth:  0.0 0.0 0.0 0.0

ADSL2
Growth:  0.0 0.0 0.0 0.0

ADSS
Growth:  0.0 0.0 0.0 0.0

AGPAT1
Growth:  0.0 0.0 0.0 0.0

AICART
Growth:  0.0 0.0 0.0 0.0

ASNS1
Growth:  0.0 0.0 0.0 0.0

ASPCTr
Growth:  0.0 0.0 0.0 0.0

C14STRr
Growth:  0.0 0.0 0.0 0.0

C4STMO1r
Growth: 

['EX_arg_L_b__C',
 'EX_biomass_c__C',
 'EX_his_L_b__C',
 'EX_ile_L_b__C',
 'EX_inost_b__C',
 'EX_leu_L_b__C',
 'EX_lys_L_b__C',
 'EX_met_L_b__C',
 'EX_o2_b__C',
 'EX_phe_L_b__C',
 'EX_pi_b__C',
 'EX_thr_L_b__C',
 'EX_trp_L_b__C',
 'EX_tyr_L_b__C',
 'EX_val_L_b__C',
 '3DSPHR__C',
 'ADSL1__C',
 'ADSL2__C',
 'ADSS__C',
 'AGPAT1__C',
 'AICART__C',
 'ASNS1__C',
 'ASPCTr__C',
 'C14STRr__C',
 'C4STMO1r__C',
 'CBPS__C',
 'CDIPTr__C',
 'CLS_hs__C',
 'DHORTS__C',
 'DSAT__C',
 'FAS100COA__C',
 'FAS120COA__C',
 'FAS140COA__C',
 'FAS160COA__C',
 'FAS80COA_L__C',
 'G3PD1__C',
 'G5SADrm__C',
 'GLNS__C',
 'GLUPRT__C',
 'GPAM_hs__C',
 'IMPC__C',
 'LNSTLSr__C',
 'NICRNS__C',
 'NP1__C',
 'OMPDC__C',
 'ORPT__C',
 'P5CRm__C',
 'PGPP_hs__C',
 'PGPPT__C',
 'PPM__C',
 'PRAGSr__C',
 'PRFGS__C',
 'PSDm_hs__C',
 'RNDR1__C',
 'SERPT__C',
 'SMS__C',
 'SQLEr__C',
 'SQLSr__C',
 'TMDS__C',
 'TRDR__C',
 'r0391__C',
 'r0666__C',
 'r0781__C',
 'RE3273C__C',
 'RE3301C__C',
 'HMGCOARc__C',
 'CO2ter__C',
 'DATPtn__C',
 'DC

In [ ]:
#Plane phase

In [ ]:
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

from time import time

from cobra.flux_analysis import production_envelope


In [ ]:
cs_data = cobra.flux_analysis.phenotype_phase_plane.find_carbon_sources(com)
#data.plot_matplotlib();

In [ ]:
prod_env = production_envelope(com, [pairs_med_50[0][0],pairs_med_50[0][1]], carbon_sources=cs_data)

In [75]:
reacts=['FDH__A', 'FDH__B', 'FDH__C']

t_u=0
t_l=0

for r in com.reactions:
    if 'FDH__' in r.id:
        t_u=r.upper_bound
        t_l=r.lower_bound
        r.upper_bound = 0
        r.lower_bound = 0
        try:
            sol_i = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
            print(r.id)
            print('Zero bounds: ', r.lower_bound,r.upper_bound)
            
            if sol_i.growth_rate < goal*0.95 or sol_i.members.growth_rate['A']< goalA*0.95 or sol_i.members.growth_rate['B']< goalB*0.95 or sol_i.members.growth_rate['C']< goalC*0.95:
                temp = r.id.split('__')
                if '___' in r.id:
                    temp[0]+='_'
                    temp[1]=temp[1][1:]

                print('Growth: ', sol_i.growth_rate, sol_i.members.growth_rate['A'], sol_i.members.growth_rate['B'], sol_i.members.growth_rate['C'])
                print('Original flux: ', sol.fluxes[temp[0]][temp[1]])
                print('Flux: ', sol_i.fluxes[temp[0]][temp[1]])
                print('Original bounds: ',t_l,t_u, end='\n--------------------------------------------------->\n\n')
            else:
                print('Original bounds: ',t_l,t_u, end='\n-----------------------------\n\n')
        except:
            print('Error',temp)
            
        r.upper_bound = t_u
        r.lower_bound = t_l

FDH__A
Zero bounds:  0 0
Growth:  0.04331404913491664 0.0 0.040127553519428046 0.05324834380381367
Original flux:  0.0001853264021832582
Flux:  0.0
Original bounds:  0.0 100.0
--------------------------------------------------->

FDH__C
Zero bounds:  0 0
Growth:  0.04331404913493348 0.0 0.040127553519744084 0.053248343803607955
Original flux:  0.0010657647844483007
Flux:  0.0
Original bounds:  0.0 100.0
--------------------------------------------------->

FTHFDH__C
Zero bounds:  0 0
Growth:  0.04331404913491481 0.0 0.040127553519372514 0.05324834380385199
Original flux:  0.08677084708092252
Flux:  0.0
Original bounds:  0.0 100.0
--------------------------------------------------->

